In [1]:
import numpy as np
from scipy.fftpack import dct, idct

import scipy.io
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
from scipy.interpolate import griddata

from helper_functions.helper_functions import plt3D2

import svd_model as svm

import forward_model as fm
import cv2 as cv

# for a bit of investigation into the process surrounding the SVD
#import pdb
from IPython.core.debugger import set_trace

# Field-varying forward model

In this notebook, you can upload your measured PSFs across the FoV. At each depth, the measured PSFs will be registered and the SVD parameters will be computed. These parameters (comps, weights) can be used to generate simulated measurements taking into account field varying aberrations. In our experiments, we measure an 8x8 grid (64 PSFs) at each depth and use rank 12. In this tutorial, we include a downsampled version measured on a 3x3 grid (9 PSFs) at 40 depths. 


## Load in measured PSFs

PSFs are expected to have a shape of (x,y,P,z), where P is the number of samples at a given depth, and z is the number of depths. Here PSFs have a shape of (486, 648, 9, 40).

In [2]:
# kyrollos' nonsense hardcoded path
#registered_psfs_path = '/home/kyrollos//LearnedMiniscope3D/MultiWiener/multiwienernet/data/PSFs/9_psfs/multiWienerPSFStack_40z.mat'

#my nonsense hardcoded path:
registered_psfs_path = '../data/LearnedMiniscope2D/MultiWiener/multiwienernet/data/PSFs/9_psfs/multiWienerPSFStack_40z_aligned.mat'

psf_name='multiWienerPSFStack_40z'

loaded_matfile = scipy.io.loadmat(registered_psfs_path)
loaded_matfile

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue May 11 17:48:00 2021',
 '__version__': '1.0',
 '__globals__': [],
 'multiWienerPSFStack_40z': array([[[[ 1.41666667, -0.5       , -2.33333333, ...,  0.66666667,
           -2.16666667, -2.41666667],
          [-0.5       ,  0.83333333, -0.16666667, ..., -1.41666667,
            1.        ,  1.33333333],
          [ 0.75      ,  1.83333333, -0.75      , ..., -1.33333333,
           -2.33333333, -2.91666667],
          ...,
          [-1.25      , -1.08333333, -0.16666667, ..., -2.33333333,
           -0.83333333,  1.5       ],
          [ 2.5       , -1.83333333, -0.25      , ..., -0.08333333,
           -0.25      ,  0.08333333],
          [-1.41666667, -2.5       , -2.        , ..., -1.91666667,
            1.16666667, -4.58333333]],
 
         [[-2.25      ,  2.08333333,  0.91666667, ..., -1.91666667,
            2.25      , -1.75      ],
          [ 3.25      , -4.16666667,  3.66666667, ..., -1.        ,
      

In [3]:
type(loaded_matfile)

dict

In [4]:
psfs=loaded_matfile[psf_name]
print(psfs.shape)

(486, 648, 9, 40)


## Registering the PFSs

In order to calcualte SVD model parameters, we need to register the PSFs. This registration is done at each depth. We pick the PSF sampled from the middle of the FoV as the reference image and register other PSFs to it. Here we show how this is done at one depth (z=0). If wokring with 3D PSFs, you will need to repeat this for each depth. 

In addition, if the PFSs contain some background, we implement a discrete cosine transform background subtraction. This is tuned to our PSFs and is optional. Set dct_on=True or False to enable or disable it. 

In [5]:
# register the PSFs and calculate their shifts
depth=0
ref_image_index=4
set_trace()
psfs_reg,shifts=svm.register_psfs(psfs[:,:,:,depth],psfs[:,:,ref_image_index,depth],dct_on=True)

--Return--
None
> /tmp/ipykernel_481346/317992781.py(4)<cell line: 4>()
      1 # register the PSFs and calculate their shifts
      2 depth=0
      3 ref_image_index=4
----> 4 set_trace()
      5 psfs_reg,shifts=svm.register_psfs(psfs[:,:,:,depth],psfs[:,:,ref_image_index,depth],dct_on=True)

ipdb> c
Removing background

done

registering

done registering



In [6]:
shifts

{0: [0, 0],
 1: [0, 0],
 2: [0, 0],
 3: [0, 0],
 4: [1, 1],
 5: [0, 0],
 6: [0, 0],
 7: [0, 0],
 8: [0, 0]}

In [7]:
psfs[:,:,:,depth].shape

(486, 648, 9)

In [8]:
psfs[:,:,ref_image_index,depth].shape

(486, 648)

In [9]:
psfs_reg.shape

(486, 648, 9)

## Calculating SVD model parameters

Using the registered PSFs and the shifts, we can now perfrom SVD up to a chosen rank less thank the number of image sampled per depth. Here we have 9 images per depth, so we chose a rank of 4. The higher the rank, the better the model but the slower the FISTA SVD algorithm. 

For a small experiment, I will fabricate shifts for these PSFs and see if I can reproduce Laura's "cloud" simulation from an image of ones

In [10]:
# shifts pulled out of thin air. Assume I cut the image into a 3x3 grid.
made_up_shifts_3x3 = {}
for x_ind in range(3):
    for y_ind in range(3):
        made_up_shifts_3x3[3*x_ind + y_ind] = [54*y_ind, 72*x_ind]

In [ ]:
# rank=4
rank=8
set_trace()
#comps, weights_interp=svm.calc_svd(psfs_reg,shifts,rank)
comps, weights_interp=svm.calc_svd(psfs_reg,made_up_shifts_3x3,rank)

--Return--
None
> /tmp/ipykernel_481346/584528219.py(3)<cell line: 3>()
      1 # rank=4
      2 rank=8
----> 3 set_trace()
      4 #comps, weights_interp=svm.calc_svd(psfs_reg,shifts,rank)
      5 comps, weights_interp=svm.calc_svd(psfs_reg,made_up_shifts_3x3,rank)

ipdb> psfs_reg.shape
(486, 648, 9)


In [ ]:
weights_interp.shape

In [ ]:
# weights_interp[:,:,:]

In [ ]:
# weights_interp[:,:,:]

In [ ]:
comps.shape

## Let's visualize the weights and components


In [ ]:
plt3D2(comps)

In [ ]:
plt3D2(weights_interp)

## simulate data using forward model

In [ ]:
#make the shape, xyzr or xyr if 2D
# Normalize weights to have maximum sum through rank of 1
weights_norm = np.sum(weights_interp[weights_interp.shape[0]//2-1,weights_interp.shape[1]//2-1,:],0).max()
weights = weights_interp/weights_norm;

#normalize by norm of all stack. Can also try normalizing by max of all stack or by norm of each slice
h=comps/np.linalg.norm(np.ravel(comps))

In [ ]:
h_dict = {"array_out": h}
scipy.io.savemat("../data/Mine_from_Waller_PSF_h.mat", h_dict)
weights_dict = {"array_out": weights}
scipy.io.savemat("../data/Mine_from_Waller_PSF_weights.mat", weights_dict)

In [ ]:
h.shape

In [ ]:
np.sum(weights_interp[weights_interp.shape[0]//2-1,weights_interp.shape[1]//2-1,:],0)

In [ ]:
weights_interp[weights_interp.shape[0]//2-1,weights_interp.shape[1]//2-1,:]

In [ ]:
# padded values for 2D
ccL = np.shape(h)[1]//2
ccU = 3*np.shape(h)[1]//2
rcL = np.shape(h)[0]//2
rcU = 3*np.shape(h)[0]//2

H=np.ndarray((np.shape(h)[0]*2,np.shape(h)[1]*2,np.shape(h)[2]), dtype=complex)
Hconj=np.ndarray((np.shape(h)[0]*2,np.shape(h)[1]*2,np.shape(h)[2]),dtype=complex)
for i in range (np.shape(h)[2]):
        H[:,:,i]=(np.fft.fft2(fm.pad2d(h[:,:,i])))
        Hconj[:,:,i]=(np.conj(H[:,:,i]))

In [ ]:
x=(plt.imread('../data/MultiWienerNet/data/hydra3.jpg'))
# x=(plt.imread('/home/kyrollos/MultiWienerNet/data/hydra3.jpg'))
x=cv.resize(x,(h.shape[1],h.shape[0]))
x=np.mean(x,2)

In [ ]:
sim=fm.A_2d_svd_crop(fm.pad2d(x),H,weights,fm.pad2d,[rcL,rcU,ccL,ccU],mode='shift_variant')

In [ ]:
plt.imshow(sim)
plt.title('Simulated Measurement')

Now, the real question is whether the SVD done by the code above produces the same thing that the authors provided to us via Google Drive. I hereby test this. First with the explicitly-called `H` and `weights`

In [ ]:
import sys
sys.path.append("../tensorflow")
import forward_model_tf as fm_tf

H_loaded,weights_loaded,_ =fm_tf.load_weights(h_path='../data/SVD_2_5um_PSF_5um_1_ds4_dsz1_comps_green_SubAvg.mat',
                                      weights_path='../data/SVD_2_5um_PSF_5um_1_ds4_dsz1_weights_green_SubAvg.mat') 

In [ ]:
len(weights_loaded[0,:,0])

In [ ]:
uniquevals = []
#for val in weights_loaded[:,:,0].reshape((314928)):
for val in weights_loaded[0,:,0]:
    to_add = True
    for compval in uniquevals:
        if val == compval:
            to_add=False
    if to_add:
        uniquevals.append(val)
        print("adding: ", val)

In [ ]:
len(uniquevals)

In [ ]:
H==H_loaded

In [ ]:
H.shape

In [ ]:
H_loaded.shape

In [ ]:
plt.imshow(x)

In [ ]:
crop_indices = [rcL,rcU,ccL,ccU]

In [ ]:
sim2 = fm_tf.sim_data(x,H,weights,crop_indices)

need `crop_indices`... what are those? They are generated in `forward_model_tf`. **TODO** examine `forward_model_tf` for how it loads weights etc. and what it actually returns.

In [ ]:
plt.imshow(sim2)

In [ ]:
sim==sim2

In [ ]:
sim

In [ ]:
sim2

`sim2` has smaller values than `sim`, as though `sim2` is normalized... 

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,15))
ax[0].imshow(sim)
ax[1].imshow(sim2)

I mean... seems good enough. Image on the right uses the tensorflow forward model which I would use in general. It has clearer features and smaller values.

### Checking the load_model post-processing
Now, let's actually compare in one go the stuff that we've done here to generate the `H` and `weights`, and what is done by `fm_tf.load_model` after actually loading the files. To do that I'll just copy-paste in the function from there, modify it to take `h` and `weights` generated by `calc_svd`. Then I'll compare the output of that to the `H`, `crop_indices` and `weights` generated.

In [ ]:
def pad2d (x):
    Ny=np.shape(x)[0]
    Nx=np.shape(x)[1]
    return np.pad(x,((Ny//2,Ny//2),(Nx//2,Nx//2)),'constant', constant_values=(0))

def pad4d(x):
    Ny=np.shape(x)[0]
    Nx=np.shape(x)[1]
    return np.pad(x,((Ny//2,Ny//2),(Nx//2,Nx//2),(0,0),(0,0)),'constant', constant_values=(0))

def crop4d(x,rcL,rcU,ccL,ccU):
    return x[rcL:rcU,ccL:ccU,:,:]

def crop2d(x,rcL,rcU,ccL,ccU):   
    return x[rcL:rcU,ccL:ccU]

def nocrop(x):
    return x

def nopad(x):
    return x

def A_2d_svd(x,H,weights,pad,mode='shift_variant'): #NOTE, H is already padded outside to save memory
    x=pad(x)
    Y=np.zeros((np.shape(x)[0],np.shape(x)[1]))
        
    if (mode =='shift_variant'):
        for r in range (0,np.shape(weights)[2]):
            X=np.fft.fft2((np.multiply(pad(weights[:,:,r]),x)))
            Y=Y+ np.multiply(X,H[:,:,r])
    
    return np.real((np.fft.ifftshift(np.fft.ifft2(Y))))

def A_2d(x,psf,pad):
    X=np.fft.fft2((pad(x)))
    H=np.fft.fft2((pad(psf)))
    Y=np.multiply(X,H)
    
    return np.real((np.fft.ifftshift(np.fft.ifft2(Y))))

def A_2d_adj_svd(Hconj,weights,y,pad):
    y=pad(y)
    x=np.zeros((np.shape(y)[0],np.shape(y)[1]))
    for r in range (0, np.shape(weights)[2]):
        x=x+np.multiply(pad(weights[:,:,r]),(np.real(np.fft.ifftshift(np.fft.ifft2(np.multiply(Hconj[:,:,r], np.fft.fft2((y))))))))
    #note the weights are real so we dont take the complex conjugate of it, which is the adjoint of the diag 
    return x

def A_2d_adj(y,psf,pad):
    H=np.fft.fft2((pad(psf)))
    Hconj=np.conj(H)
    x=(np.real(np.fft.ifftshift(np.fft.ifft2(np.multiply(Hconj, np.fft.fft2((pad(y))))))))
    
    return x

def A_3d(x,h,pad):
    #h is the psf stack
    #x is the variable to convolve with h
    x=pad(x)
    B=np.zeros((np.shape(x)[0],np.shape(x)[1]))
        

    for z in range (0,np.shape(h)[2]):
        #X=np.fft.fft2((np.multiply(pad(weights[:,:,z]),x)))
        B=B+ np.multiply(np.fft.fft2(x[:,:,z]),np.fft.fft2(pad(h[:,:,z])))
    
    return np.real((np.fft.ifftshift(np.fft.ifft2(B))))


def A_3d_svd(v,alpha,H,pad):
    #alpha is Ny-Nx-Nz-Nr, weights
    #v is Ny-Nx-Nz
    #H is Ny-Nx-Nz-Nr
    # b= sum_r (sum_z (h**alpra.*v))
    b=np.zeros((np.shape(v)[0],np.shape(v)[1]))
    for r in range (np.shape(H)[3]):
        for z in range (np.shape(H)[2]):
            b=b+np.multiply(H[:,:,z,r],np.fft.fft2(np.multiply(v[:,:,z],alpha[:,:,z,r])))
    
    return np.real(np.fft.ifftshift(np.fft.ifft2(b)))

def A_3d_adj(x,h,pad):
    y=np.zeros(np.shape(h))
    X=np.fft.fft2(pad(x))
    for z in range(np.shape(h)[2]):
        H=np.conj(np.fft.fft2(pad(h[:,:,z])))
        y[:,:,z]=np.real(np.fft.ifftshift(np.fft.ifft2(np.multiply(H,X))))
    return y

def A_3d_adj_svd(b,alpha,Hconj,pad):
    #y=sum_r(alpha.*H_conj**b)
    y=np.zeros((np.shape(alpha)[0],np.shape(alpha)[1],np.shape(alpha)[2]))
    B=np.fft.fft2(pad(b))
    for z in range(np.shape(alpha)[2]):
        for r in range(np.shape(alpha)[3]):
            y[:,:,z]=y[:,:,z]+np.multiply(alpha[:,:,z,r],np.fft.ifftshift(np.fft.ifft2(np.multiply(B,Hconj[:,:,z,r]))))
        
    return y

def grad(v):
    return np.array(np.gradient(v))  #returns gradient in x and in y


def grad_adj(v):  #adj of gradient is negative divergence
    z = np.zeros((n,n)) + 1j
    z -= np.gradient(v[0,:,:])[0]
    z -= np.gradient(v[1,:,:])[1]
    return z

def load_weights_postprocess(h, weights):
    # we don't need to load the files
    #h=scipy.io.loadmat(h_path) 
    #weights=scipy.io.loadmat(weights_path )

    # depth_plane is not used, since h and weights are 3D rather than 4D
    depth_plane=0 #NOTE Z here is 1 less than matlab file as python zero index. So this is z31 in matlab

    # again, no need do access data of what would have been a dict
    #h=h['array_out']
    #weights=weights['array_out']
    
    # we aren't swapping anything, since it's already (x,y,r)
    ## make sure its (x,y,z,r)
    #h=np.swapaxes(h,2,3)
    #weights=np.swapaxes(weights,2,3)

    #h=h[:,:,depth_plane,:]
    #weights=weights[:,:,depth_plane,:]

    # Normalize weights to have maximum sum through rank of 1
    weights_norm = np.max(np.sum(weights[np.shape(weights)[0]//2-1,np.shape(weights)[1]//2-1,:],0))
    weights = weights/weights_norm;

    #normalize by norm of all stack. Can also try normalizing by max of all stack or by norm of each slice
    h=h/np.linalg.norm(np.ravel(h))

    # padded values for 2D

    ccL = np.shape(h)[1]//2
    ccU = 3*np.shape(h)[1]//2
    rcL = np.shape(h)[0]//2
    rcU = 3*np.shape(h)[0]//2

    H=np.ndarray((np.shape(h)[0]*2,np.shape(h)[1]*2,np.shape(h)[2]), dtype=complex)
    Hconj=np.ndarray((np.shape(h)[0]*2,np.shape(h)[1]*2,np.shape(h)[2]),dtype=complex)
    for i in range (np.shape(h)[2]):
        H[:,:,i]=(np.fft.fft2(pad2d(h[:,:,i])))
        Hconj[:,:,i]=(np.conj(H[:,:,i]))
    return H,weights,[rcL,rcU,ccL,ccU]

In [ ]:
H_post, weights_post, croppings_post = load_weights_postprocess(h, weights)

In [ ]:
croppings_post==crop_indices

From above, it appears that **all you have to do is perform:** 
```python
psfs_reg,shifts=svm.register_psfs(psfs[:,:,:,depth],psfs[:,:,ref_image_index,depth],dct_on=True)
comps, weights_interp=svm.calc_svd(psfs_reg,shifts,rank)
#make the shape, xyzr or xyr if 2D
# Normalize weights to have maximum sum through rank of 1
weights_norm = np.sum(weights_interp[weights_interp.shape[0]//2-1,weights_interp.shape[1]//2-1,:],0).max()
weights = weights_interp/weights_norm;

#normalize by norm of all stack. Can also try normalizing by max of all stack or by norm of each slice
h=comps/np.linalg.norm(np.ravel(comps))

H, weights, crop_indices = load_weights_postprocess(h, weights)
```

And then `H`, `weights`, `crop_indices` can be passed to `fm_tf.sim_data` with no problem.

In [ ]:
H_post.shape

In [ ]:
weights_post.shape

In [ ]:
croppings_post

In [ ]:
scipy.io.loadmat("../data/MultiWienerNet/data/SVD_2_5um_PSF_5um_1_ds4_dsz1_comps_green_SubAvg.mat")